<p><img style="padding: 0 15px; float: left;" src="Data/Logo.png"
  alt="PreLease Logo" Width='140' Height= '250'/></p> 
<p> <h3> PreLease Minneapolis Housing Workbook</h3>
<p> <h3> By Abu Nayeem (Programmer & Co-Founder)</h3>

The underlying datasets are Hennepin Parcel Assessor Data and Minneapolis Rental Licenses Data. County Parcel Data was not up-to-date, so not used

In [4]:
#Import Libraries & Load Data
import plotly.express as xp   # now it works
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests
import folium
import seaborn as sns
import json
from urllib.request import urlopen


#Load 2021 Assessor Parcel Data; #Last Update 202111
df = pd.read_csv('Data/Raw_Assessors_Parcel_Data_2021.csv')
#Source: https://opendata.minneapolismn.gov/datasets/assessors-parcel-data-2021/explore

#download from API



df.head(4)

,ObjectId,PIN,ASMTYEAR,TAXYEAR,HOUSE_NO,STREET_NAME,UNIT_NO,ZIP_POSTAL,FORMATTED_ADDRESS,OWNERNM,...,NUM_STORIES,NUM_GAR_STALLS,PRIMARYHEATING,CONSTRUCTIONTYPE,EXTERIORTYPE,ROOF,TOTAL_UNITS,FIREPLACES,BATHROOMS,BEDROOMS
0,1,102824110001,2021,2022,3033 1/2,29TH AVE S,NaN,55406,3033 1/2 29TH AVE S,CITY OF MINNEAPOLIS ET AL,...,NaN,0,NaN,NaN,NaN,NaN,0.0,0,0,0
1,2,102824110004,2021,2022,3033,29TH AVE S,NaN,55406-1919,3033 29TH AVE S,KATHERINE ZOBRO & A DANIELS,...,1.5,2,FORCED AIR,WOOD FRAME,OTHER,GABLE,1.0,0,1,3
2,3,102824110005,2021,2022,3106,28TH AVE S,NaN,55406-1918,3106 28TH AVE S,DARLENE L CHAPMAN ET AL,...,1.5,1,GRAVITY,WOOD FRAME,METAL/VINYL,GABLE,1.0,1,1,3
3,4,102824110006,2021,2022,3108,28TH AVE S,NaN,55406-1918,3108 28TH AVE S,J B STENGEL & S L HEIDEMANN,...,1.5,2,HOT WATER,WOOD FRAME,STUCCO,GABLE,2.0,1,2,3


### 2021 Assessor Parcel Data; Excluded Cols and Notes
- Snippets of address, 
- Columns [Taxpayer1-4] provides taxpayer AND address. The address category is not filled consistently to use this information; the active rental licenses would be reliable
- Some excluded categories include Landvalue, number of garages, below and above ground area of property, roof, fireplace
- The "Total Units" does NOT equate to number of rental units
- The __unique identifier__ is __PIN__, which is associated with unique taxcode


In [2]:


#Select Columns of Interest
p=['PIN',  'ZIP_POSTAL', 'FORMATTED_ADDRESS','OWNERNM','NEIGHBORHOOD', 'COMMUNITY','WARD', 'ZONING', 'LANDUSE', 'PARCEL_AREA_SQFT', 'PRIMARY_PROP_TYPE', 'BLDGVALUE', 'TOTALVALUE', 'IS_MULTIPLE_USES', 'IS_EXEMPT', 
   'NUM_BLDGS','BUILDINGUSE','YEARBUILT', 'NUM_STORIES', 'NUM_GAR_STALLS', 'PRIMARYHEATING','CONSTRUCTIONTYPE','EXTERIORTYPE','TOTAL_UNITS','BATHROOMS','BEDROOMS']
a=df[p]

#rename columns
cols=['TID','Zip', 'address','Owner','Neighborhood', 'Community', 'Ward','Zoning','LandUse','Area','Property_Type', 'BLD_Val', 'Total_Val', 
     'Mult_Use','Is_Exempt','Num_Bldgs','Bldg_Use','YearBuilt','Num_Stories','Num_Garage','Heating','Construction','Exterior','Tot_Units', 'Bathrooms','Bedrooms']
a.columns= cols

#Column Edits
a['address']= a['address'].str.strip() #remove whitespace
a['address']= a['address'].str.title()
a['Zip5'] = a['Zip'].str[:5] #first 5 digits
a['Community']= a['Community'].str.title() #Capitalization & Lowercase
a['Community']= a['Community'].str.replace('-',' ') 
a['Neighborhood']= a['Neighborhood'].str.title() 
a['Owner']= a['Owner'].str.title()
a['LandUse']= a['LandUse'].str.title()

# Create Simplifed Zoning Varaible;  https://www2.minneapolismn.gov/business-services/planning-zoning/zoning-maps/zoning-code-descriptions/ 
def zone(x): 
    if x=='R1':
        return 'Single_LowDensity_Residential'
    elif x=='R2':
        return 'Two_LowDensity_Residential'
    elif x in ('R3','R4'):
        return 'Mult_MedDensity_Residential'   
    elif x in ('R5','R6'):
        return 'Mult_HighDensity_Residential' 
    elif x in ('C1','C2','C3','C4'):
        return 'Commerical'
    elif x in ('I1','I2','I3','I4'):
        return 'Industrial'
    elif x=="OR":
        return 'Office_Residential'
    elif x=='B4':
        return 'Downtown'    
    else:
        return 'Missing'    

a['Zoning1']= a['Zoning'].str[:2] #keep first two strings
a['Zoning1']= a['Zoning1'].apply(zone) #run Function  

# the neighborhood geoson file does not separate the Phillips and put them as one
a['Neighborhood']= a['Neighborhood'].str.replace('Phillips West','Phillips') 
a['Neighborhood']= a['Neighborhood'].str.replace('Midtown Phillips','Phillips')
a['Neighborhood']= a['Neighborhood'].str.replace('East Phillips','Phillips')

a.head(2)
#a.query('Add_Unit=="P3217"')
#a.Zoning1.unique()

,TID,Zip,address,Owner,Neighborhood,Community,Ward,Zoning,LandUse,Area,...,Num_Stories,Num_Garage,Heating,Construction,Exterior,Tot_Units,Bathrooms,Bedrooms,Zip5,Zoning1
0,102824110001,55406,3033 1/2 29Th Ave S,City Of Minneapolis Et Al,Longfellow,Longfellow,2.0,R1A,Vacant Land,2145.0,...,NaN,0,NaN,NaN,NaN,0.0,0,0,55406,Single_LowDensity_Residential
1,102824110004,55406-1919,3033 29Th Ave S,Katherine Zobro & A Daniels,Longfellow,Longfellow,2.0,R2B,Single-Family Detached Dw,7811.0,...,1.5,2,FORCED AIR,WOOD FRAME,OTHER,1.0,1,3,55406,Two_LowDensity_Residential


###  2021 Active Rental License Data; Excluded Cols and Notes
- [Source](https://opendata.minneapolismn.gov/datasets/cityoflakes::active-rental-licenses/about)
- <font color=blue>X and Y</font> are replacement Longitude and Latitude 
Columns 
- The __Assessor Table__ has more reliable neighborhood information.
- There are __multiple APN__ when they are suppose to be __unique identifier__ to tax code
- The __LicenseNumber__ is unique (4 duplicate), but has no common key to connect tables
- Excluded WebMercator, policeprecinct code, ton of info on Owner (Address, Zip, etc), we are mostly interested in the __Applicant__ as they are managing the property 
- the __category__ variable is still unknown

In [3]:
#Loading Active Rental License Data

#Load Data
#ef = pd.read_csv('Data/Raw_Active_Rental_Licenses.csv')

#Load from API
json_file = "https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Active_Rental_Licenses/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"
data = urlopen(json_file).read()
raw_json = json.loads(data)
formatted_json = [feature['attributes'] for feature in raw_json['features']]
ef = pd.DataFrame(formatted_json)

#change order
p=['X','Y', 'apn','category','status','licenseNumber','milestone','tier','issueDate', 'expirationDate', 'address', 'ownerName', 'ownerAddress1', 
   'ownerAddress2', 'ownerPhone', 'ownerEmail', 'applicantName', 'applicantAddress1', 'applicantPhone', 'applicantEmail', 'licensedUnits']       
ren=ef[p]
cols= ['longitude','latitude', 'TID','category','status','licenseNumber','milestone','tier','issueDate', 'expirationDate', 'address', 'ownerName', 'ownerAddress1',
       'ownerAddress2',  'ownerPhone', 'ownerEmail', 'applicantName', 'applicantAddress1', 'applicantPhone', 'applicantEmail', 'licensedUnits']
ren.columns= cols


from datetime import datetime

ren['address']= ren['address'].str.title()
ren['ownerAddress1']= ren['ownerAddress1'].str.title() #will be used for matching
ren['applicantAddress1']= ren['applicantAddress1'].str.title()
ren['applicantName']= ren['applicantName'].str.title()
ren['ownerName']= ren['ownerName'].str.title()

#We will be using the applicant email and phonenumber
# if they are EMPTY then we are filling them with owner's Info
ren['applicantPhone']= np.where(ren.applicantPhone.isnull(), ren['ownerPhone'] , ren['applicantPhone']) 
ren['applicantEmail']= np.where(ren.applicantEmail.isnull(), ren['ownerEmail'] , ren['applicantEmail']) 

#ren['ownerPhone'].query(applicantPhone.isnull())


# Owner-Occupied Strategy; if property address equals the owner address OR applicant address
#Shorten all address
ren['add6']= ren['address'].str[:6]
ren['oadd6']= ren['ownerAddress1'].str[:6]
ren['padd6']= ren['applicantAddress1'].str[:6]

#Owner Occupied Column
ren['Owner_Occupied1'] = np.where(ren['add6'] == ren['oadd6'], 1, 0) # if true, then Value One
ren['Owner_Occupied2'] = np.where(ren['add6'] == ren['padd6'], 1, 0)
ren['Owner_Occupied3']= ren['Owner_Occupied1'] + ren['Owner_Occupied2']
ren['Owner_Occupied'] = np.where(ren['Owner_Occupied3']>0,'Yes' , 'No')

#DateTime Variables
ren['ExpDate']= pd.to_datetime(ren['expirationDate']) # Create new column called DateTime
ren['ExpYear']= ren['ExpDate'].dt.year #create year column
ren['ExpMonth']= ren['ExpDate'].dt.month
ren['ExpDay']= ren['ExpDate'].dt.day
ren['LicExpDate']='03/01/' + ren['ExpYear'].astype(str)

ren['IssDate']= pd.to_datetime(ren['issueDate']) # Create new column called DateTime
ren['IssYear']= ren['IssDate'].dt.year #create year column
ren['IssMonth'] = ren['IssDate'].dt.month # Create Month Category
ren['IssDay'] = ren['IssDate'].dt.day #Create Day of the Current month
ren['LicIssueDate']= ren['IssMonth'].astype(str) + '/' + ren['IssDay'].astype(str) + '/' + ren['IssYear'].astype(str)

#Duplicate Licenses
# LIC386276,LIC389668, LIC396985, LIC397106


#There are 384 EXPIRED LEASES
bad= ren.query('ExpYear<2021') #outdate records
bad.to_csv(r'Error_Log/Rental_ExpiredEntries.csv',index=False)
ren= ren.query('ExpYear>2021') #select active leases

#TIDs are not suppose to have letters
bad1=ren[ren['TID'].str.contains('A')]
bad1.to_csv(r'Error_Log/Rental_TID_A_Entries.csv',index=False)
ren= ren[~ren['TID'].str.contains('A')]
ren['TID'] = ren['TID'].astype(np.int64) #change datatype

ren.head(2)


,longitude,latitude,TID,category,status,licenseNumber,milestone,tier,issueDate,expirationDate,...,ExpDate,ExpYear,ExpMonth,ExpDay,LicExpDate,IssDate,IssYear,IssMonth,IssDay,LicIssueDate
0,-93.307754,44.895922,2102824320092,CONV,Active,LIC385097,Active,Tier 1,2019/11/07 10:03:49+00,2023/03/01 00:00:00+00,...,2023-03-01 00:00:00+00:00,2023,3,1,03/01/2023,2019-11-07 10:03:49+00:00,2019,11,7,11/7/2019
1,-93.290564,45.046278,1211821240019,CONV,Active,LIC375543,Active,Tier 1,2019/01/07 08:11:35+00,2023/03/01 00:00:00+00,...,2023-03-01 00:00:00+00:00,2023,3,1,03/01/2023,2019-01-07 08:11:35+00:00,2019,1,7,1/7/2019


In [5]:
ef.columns

Index(['X', 'Y', 'apn', 'OBJECTID', 'licenseNumber', 'category', 'milestone',
       'tier', 'status', 'issueDate', 'expirationDate', 'address', 'ownerName',
       'ownerAddress1', 'ownerAddress2', 'ownerCity', 'ownerState', 'ownerZip',
       'ownerPhone', 'ownerEmail', 'applicantName', 'applicantAddress1',
       'applicantAddress2', 'applicantCity', 'applicantState', 'applicantZip',
       'applicantPhone', 'applicantEmail', 'licensedUnits', 'ward',
       'neighborhoodDesc', 'communityDesc', 'policePrecinct',
       'shortTermRental', 'latitude', 'longitude', 'xWebMercator',
       'yWebMercator'],
      dtype='object')

## Matching Strategy and Prep

- On the Active Rental Licenses, there are __multiple TIDS__. My strategy is to split the data into two; Single ID and Multiple ID. 
- For the single ID, match on __TID__.
- For the multiple ID, FIRST match on __TID AND address__ to lock a unique value to the Parcel table.
- Second for remainder unmatched, match on __address__ . For this matched group, I retrieve the **correct** corresponding TID from the Parcel table, which will update the initial PID.
- The common linking table will involve TID, licensenumber, to assure uniqueness on active rental. When linking tables, I want few columns as possible
- Some of the non-linked single entries could be the result of rental properties being new and an assessment has not been made

In [10]:

#Setup the Duplicate test on Rental Licenses
ren['dex']=1 
p=['dex', 'TID']

h=ren[p].groupby(['TID']).sum()
#h.dex.unique()
Sin=h.query('dex==1').reset_index() # All TIDs that had a single record
Mult=h.query('dex>1').reset_index() # ALL Tids that had mult record

#this table excludes all values that did not match
S1= pd.merge(ren, Sin, on='TID', how='right') #S1.shape #21166 records
M1= pd.merge(ren, Mult, on='TID', how='right') #742 records

#Error Table of Multiple TIDs on Active Rental License
M1.to_csv(r'Error_Log/Rental_MultTID_Entries.csv',index=False)


#Testing Cdes
#p=['apn','category', 'licenseNumber','issueDate', 'expirationDate', 'address', 'ownerName', 'ownerAddress1', 'ownerAddress2', 'ownerState', 'ownerPhone', 'ownerEmail', 'applicantName', 'applicantPhone', 'applicantEmail']       
#reny=ef[p]
#cols= ['TID', 'category','licenseNumber','issueDate', 'expirationDate', 'address', 'ownerName', 'ownerAddress1', 'ownerAddress2', 'ownerState', 'ownerPhone', 'ownerEmail', 'applicantName', 'applicantPhone', 'applicantEmail']   
#reny.columns= cols


#-93.307754	44.895922	2102824320092
#reny.query("licenseNumber=='LIC395570'")
#reny.query("TID=='0102824110007'")
#a.query("TID=='102824110007'")
#ren["TID"].str.find('B')
##ren.query(X)

#a[a['address'].str.contains('4815 28TH AVE', na=False)]

#LIC394538
#LIC399856
#1302824140210

In [5]:
#The Whittle algorithm, find patterns and cut the number of unlinked entries each session

#SINNNNGLLLEEE DataSET

#Prepare Parcel Data
dols= ['address','Community','TID']
Hen=a[dols]
#Hen['index'] = range(1, len(Hen) + 1) #create Index variable

#Prepare Rental Data
test=['LicExpDate', 'address','TID','licenseNumber']
m= S1[test]

#Merge and the null list shows what are left unmatched
Mas1= pd.merge(Hen, m, on='TID', how='right')
Mas1[Mas1['address_x'].isnull()] #141 missing hits; address_X is from Parcel data
k1=Mas1[Mas1['address_x'].notnull()] #Save matched hits
#Export Error Csv
Mas1[Mas1['address_x'].isnull()].to_csv(r'Error_Log/Single_TID_Non-Match_Entries.csv',index=False)


#MULTIPLEEE DATASET

#Step1 
#Prepare Rental Data
test=['LicExpDate', 'address','TID', 'category','licenseNumber']
m= M1[test]
m=m.query("category!='ShrtTrmLic'") # ShortTermLease cause several duplicates so exclude

#Merge and the null list shows what are left unmatched
Mas2= pd.merge(Hen, m, on=['TID','address'], how='right') #we matched on TID and address
MW=Mas2[Mas2['Community'].notnull()] # 185 filled and 495 left

#Step2; From the 495 remaining, Match now only on address

ly=Mas2[Mas2['Community'].isnull()] 
test1=['LicExpDate', 'address','TID','licenseNumber']
Mas3= pd.merge(Hen, ly, on='address', how='right')
wy=Mas3[Mas3['Community_x'].notnull()] #193 matched
wy.to_csv(r'Error_Log/Incorrect_TIDx_Entries.csv',index=False) #Create error report, this should be corrected by government
wy['TID']=wy['TID_x'] #this replaces the TID from the rental table with the correct TID from parcel data


#Creating the Connect Table 
plug=['TID','licenseNumber']
key1= k1[plug] #Single
key2=MW[plug] #Mult [TID & address]
key3=wy[plug] #Mult [address] with updated TID

KFin=pd.concat([key1,key3,key2], ignore_index=True) #combine values
KFin=KFin.drop_duplicates() #remove duplicates
KFin.shape


#KFin['dex']=1
#p=['dex', 'TID']

#h=KFin[p].groupby(['TID']).sum()
#h.dex.unique()
#h.query('dex>1')


(21403, 2)

In [11]:
#pd.merge(Hen, ly, on='address', how='right')

In [12]:
#Final Merge

#Pre-Merge: Selecting Tables of Interest

#clean for data analysis
cl=['longitude','latitude','category', 'TID','licenseNumber','tier','LicExpDate', 'LicIssueDate', 'address', 'ownerName',
    'applicantName', 'licensedUnits', 'latitude', 'longitude','Owner_Occupied', 'applicantPhone', 'applicantEmail','applicantAddress1','ownerAddress1']       
rc=ren[cl] #Rental License


cols=['TID','Zip5','Owner','Neighborhood', 'Community', 'Ward','Zoning','LandUse','Area','Property_Type', 'BLD_Val', 'Total_Val', 
     'Mult_Use','Is_Exempt','Num_Bldgs','Bldg_Use','YearBuilt','Num_Stories','Heating','Construction','Exterior','Tot_Units', 'Bathrooms','Bedrooms', 'Zoning1']
Pl=a[cols] #Parcel 
 

#columns to export for Prelease
h= ['TID', 'address', 'Zip5','YearBuilt','Area','Num_Bldgs','Total_Val','Tot_Units','licensedUnits','tier','LandUse',
    'Community','Neighborhood','Ward','Owner','ownerName','applicantName','applicantPhone', 'applicantEmail',
    'Heating','Bathrooms','Bedrooms','longitude','latitude','Zoning1' ]

#Final Merging

MS=pd.merge(KFin, rc, on='licenseNumber', how='right') #Rental table connect to linking table by Licensenumber
#MS[MS['TID_x'].isnull()] #505 total missing recprd
MS['TID']=MS['TID_x'] #Update the TID with the linking tabel TID
MS=MS[MS['TID_x'].notnull()] # remove null values 
MS1=pd.merge(Pl,MS , on='TID', how='right') #Finally, connect to Parcel table; merging is complete

MS1.shape

MS1[MS1['Ward'].isnull()]
MS1['Units_Percent']=MS1['licensedUnits']
MS1['Index']=1

MS1[h].to_csv(r'Data/Mpls_Post.csv',index=False) #this table is save for loading
MS1.to_csv(r'Data/Mpls_Report.csv',index=False) # This will be the reporting csv




#Fill in the missing community variables from the Renter data to those not matched in Parcel
#e= MS1[MS1['Neighborhood'].notnull()]
#d= MS1[MS1['Neighborhood'].isnull()]
#d['Ward']= d['ward']
#d['Neighborhood']= d['neighborhoodDesc']
#d['Community']= d['communityDesc']
#only 50 records missing

#MS1=pd.concat([e,d], ignore_index=True)
#MS1.shape



In [28]:


reny.query("TID=='102824110007'")


,longitude,latitude,TID,category,status,licenseNumber,milestone,tier,issueDate,expirationDate,...,ExpYear,ExpMonth,ExpDay,LicExpDate,IssDate,IssYear,IssMonth,IssDay,LicIssueDate,dex
17531,-93.231951,44.946362,102824110007,CHOWN,Active,LIC365747,License Re,Tier 1,2018/03/30 08:06:58+00,2022/03/01 00:00:00+00,...,2022,3,1,03/01/2022,2018-03-30 08:06:58+00:00,2018,3,30,3/30/2018,1


In [35]:
#Testing Cdes
p=['apn','category', 'licenseNumber','issueDate', 'expirationDate', 'address', 'ownerName', 'ownerAddress1', 'ownerAddress2', 'ownerState', 'ownerPhone', 'ownerEmail', 'applicantName', 'applicantPhone', 'applicantEmail']       
reny=ef[p]
cols= ['TID', 'category','licenseNumber','issueDate', 'expirationDate', 'address', 'ownerName', 'ownerAddress1', 'ownerAddress2', 'ownerState', 'ownerPhone', 'ownerEmail', 'applicantName', 'applicantPhone', 'applicantEmail']   
reny.columns= cols
m=['TID','licenseNumber']
#a.query("TID=='102824110007'")
#MS1[m].query("TID=='102824110007'")
#ren.drop_duplicates(keep=False)
reny.query("licenseNumber=='LIC365747'")

,TID,category,licenseNumber,issueDate,expirationDate,address,ownerName,ownerAddress1,ownerAddress2,ownerState,ownerPhone,ownerEmail,applicantName,applicantPhone,applicantEmail
17531,0102824110007,CHOWN,LIC365747,2018/03/30 08:06:58+00,2022/03/01 00:00:00+00,3116 28TH AVE S,Vincent O Adanene,16515 Lake Ridge Dr N,NaN,MN,NaN,NaN,Vincent O Adanene,NaN,NaN


In [ ]:
#2840 28TH AVE S